In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.nn import functional as F
import torch.optim as optim
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
import math
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import time
import uuid
import optuna
import plotly
import matplotlib
from torch.utils.tensorboard import SummaryWriter
import torchmetrics
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
torch.isnan(torch.tensor(np.load("./amex_preprocessed.npz")["train_floats"])).sum().item()

In [11]:
class AmericanExpressPreprocessedProfileTimeSeriesDataset(Dataset):
    def __init__(self, dataset_file, nrows=False, transformation=False):
        data = np.load(dataset_file)
        num_data_train = data["train_floats"].reshape(data["train_floats"].shape[0], -1, 13)
        cat_data_train = data["train_cat"].reshape(data["train_cat"].shape[0], -1, 13)
        self.y = data["train_y"]

        if nrows:
            num_data_train = num_data_train[0:nrows]
            cat_data_train = cat_data_train[0:nrows]
            self.y = self.y[0:nrows]

        self.dataset = np.concatenate([num_data_train, cat_data_train], axis=1)
        self.transformation = transformation

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        data = self.dataset[idx]
        label = self.y[idx]

        data = torch.tensor(data, dtype=torch.float32, requires_grad=True)
        label = torch.tensor([label], dtype=torch.float32)
        label = label.to(device)
        data = data.to(device)
        if self.transformation: data = self.transformation(data)
        return data, label

In [12]:
class AmericanExpressPreprocessedProfileTimeSeriesDatasetValidate(Dataset):
    def __init__(self, dataset_file, nrows=False, transformation=False):
        data = np.load(dataset_file)
        num_data_test = data["test_floats"].reshape(data["test_floats"].shape[0], -1, 13)
        cat_data_test = data["test_cat"].reshape(data["test_cat"].shape[0], -1, 13)
        self.y = data["test_y"]

        if nrows:
            num_data_test = num_data_test[0:nrows]
            cat_data_test = cat_data_test[0:nrows]
            self.y = self.y[0:nrows]

        self.dataset = np.concatenate([num_data_test, cat_data_test], axis=1)
        self.transformation = transformation

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        data = self.dataset[idx]
        label = self.y[idx]

        data = torch.tensor(data, dtype=torch.float32, requires_grad=True)
        label = torch.tensor([label], dtype=torch.float32)
        label = label.to(device)
        data = data.to(device)
        if self.transformation: data = self.transformation(data)
        return data, label

In [ ]:
class Block(nn.Module):

    def __init__(self, trial, i):
        super(Block, self).__init__()

        activation = trial.suggest_categorical(f"activation_conv_layer{i}", ["LeakyReLU", "ReLU", "ELU"])

        self.activ = getattr(nn, activation)()

        padding_mode = "replicate"#trial.suggest_categorical(f"padding_mode_layer{i}", ["zeros", "reflect", "replicate"])

        kernel_size1 = trial.suggest_int(f"conv1_kernel_layer{i}", 2, 13)
        kernel_size2 = trial.suggest_int(f"conv2_kernel_layer{i}", 2, 13)
        kernel_size3 = trial.suggest_int(f"conv3_kernel_layer{i}", 2, 13)

        channel_size_conv_intermediate = 189#trial.suggest_int(f"channel_size_conv_intermediate_layer{i}", 100, 500, 50)


        self.conv1 = nn.Sequential(
            nn.Conv1d(189, channel_size_conv_intermediate, kernel_size=kernel_size1, stride=1, padding="same", padding_mode=padding_mode),
            nn.BatchNorm1d(channel_size_conv_intermediate),
        )

        self.conv2 = nn.Sequential(
            nn.Conv1d(channel_size_conv_intermediate, channel_size_conv_intermediate, kernel_size=kernel_size2, stride=1, padding="same", padding_mode=padding_mode),
            nn.BatchNorm1d(channel_size_conv_intermediate),
        )

        self.conv3 = nn.Sequential(
            nn.Conv1d(channel_size_conv_intermediate, 189, kernel_size=kernel_size3, stride=1, padding="same", padding_mode=padding_mode),
            nn.BatchNorm1d(189),
        )

    def forward(self, x):
        identity = x
        x = self.conv1(x)
        x = self.activ(x)
        x = self.conv2(x)
        x = self.activ(x)
        x = self.conv3(x)
        x = x + identity
        x = self.activ(x)
        return x

class ResNet(nn.Module):


    def __init__(self, trial):
        super(ResNet, self).__init__()

        n_layers_blocks = trial.suggest_int("n_layers_blocks", 1, 10)

        self.blocks = nn.Sequential()

        for i in range(n_layers_blocks):
            self.blocks.append(Block(trial, i))
            
        self.linear = nn.Sequential()

        linear_in = 189

        n_layers_linear = trial.suggest_int("n_layers_linear", 1, 20)


        for k in range(n_layers_linear):

            linear_out = trial.suggest_int(f"linear_size_layer{k}", 100, 300, 50)

            self.linear.append(nn.Linear(linear_in, linear_out))

            self.linear.append(nn.LeakyReLU())

            linear_in = linear_out

        self.linear.append(nn.Linear(linear_in, 1))

        self.linear.append(nn.Sigmoid())


    def forward(self, x):
        x = self.blocks(x)

        avg = nn.AvgPool1d(13, stride=1)
        x = avg(x)
        x = x.view(x.shape[0], -1)
        x = self.linear(x)
        return x

In [18]:
class ConvNet(nn.Module):
    def __init__(self, trial):
        super(ConvNet, self).__init__()

        self.trial = trial

        conv_activation = trial.suggest_categorical(f"activation_conv", ["LeakyReLU", "ReLU", "ELU"])
        self.conv_activ = getattr(nn, conv_activation)()

        lin_activation = trial.suggest_categorical(f"activation_lin", ["LeakyReLU", "ReLU", "ELU"])
        self.lin_activ = getattr(nn, lin_activation)()

        n_layers_blocks = self.trial.suggest_int("n_layers_blocks", 1, 30)
        self.blocks = nn.Sequential()
        conv_in = 178
        for i in range(n_layers_blocks):
            conv_out = self.trial.suggest_int(f"conv_size_layer{i}", 100, 300, 10)
            kernel_size = self.trial.suggest_int(f"kernel_size_layer{i}", 1, 13)
            self.blocks.append(nn.Conv1d(conv_in, conv_out, kernel_size=kernel_size, stride=1, padding="same", padding_mode="replicate"))
            self.blocks.append(self.conv_activ)
            batchnorm = self.trial.suggest_categorical(f"conv_batchnorm_layer{i}", [True, False])
            if batchnorm: self.blocks.append(nn.BatchNorm1d(conv_out))

            conv_in = conv_out


        self.linear = nn.Sequential()
        linear_in = conv_out
        n_layers_linear = self.trial.suggest_int("n_layers_linear", 1, 10)
        for k in range(n_layers_linear):
            linear_out = self.trial.suggest_int(f"linear_size_layer{k}", 100, 300, 10)
            self.linear.append(nn.Linear(linear_in, linear_out))
            self.linear.append(self.lin_activ)
            batchnorm = self.trial.suggest_categorical(f"linear_batchnorm_layer{k}", [True, False])            
            if batchnorm: self.linear.append(nn.BatchNorm1d(linear_out))

            linear_in = linear_out
        self.linear.append(nn.Linear(linear_out, 1))
        self.linear.append(nn.Sigmoid())


    def forward(self, x):
        x = self.blocks(x)
        pool = self.trial.suggest_categorical(f"pool", ["AvgPool1d", "MaxPool1d"])            
        pooling = getattr(nn, pool)(13, stride=1)
        x = pooling(x)
        x = x.view(x.shape[0], -1)
        x = self.linear(x)
        return x

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        
        self.linear = nn.Sequential(
            nn.Linear(20, 10),
            nn.ReLU(),
            nn.Linear(10, 5),
            nn.ReLU(),
            nn.Linear(5, 1),
            nn.Sigmoid()
        )
    
    def forward(self,x):
        h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        output, (hn, cn) = self.lstm(x, (h_0, c_0))
        hidden = torch.cat((hn[-2,:,:], hn[-1,:,:]), dim = 1)     
        out = self.linear(hidden)
        return out

model = LSTM(189, 10, 10)
model = model.to(device)

In [ ]:
def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:

    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)



In [28]:
def test_loop(dataloader, model, loss_fn, epoch):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            pred = torch.round(pred)
            correct += (pred == y).type(torch.float).sum().item()


    test_loss /= num_batches
    correct /= size
    #print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return test_loss

In [ ]:
def validate(dataloader, model):
    with torch.no_grad():
        for batch, (X, y) in enumerate(dataloader):
            # Compute prediction and loss
            X = X.to(device)
            y = y.to(device)
            pred = model(X)        


            pred_df = pd.DataFrame(pred.cpu().detach(), columns=["prediction"])
            target_df = pd.DataFrame(y.cpu().detach(), columns=["target"])

            uuid_df = pd.DataFrame(data={"uuid": [uuid.uuid4() for _ in range(len(pred_df.index))]})

            pred_df = pd.concat([uuid_df, pred_df], axis=1)
            target_df = pd.concat([uuid_df, target_df], axis=1)

            amex_metric_value = amex_metric(target_df, pred_df)

            return amex_metric_value

            #print(f"Amex Score (batch {batch:>1d}): {amex_metric_value:>8f} \n")

In [33]:
def train_loop(dataloader, model, loss_fn, optimizer, epoch):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        X = X.to(device)
        y = y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        #writer.add_scalar('Loss/train', np.random.random(), epoch*size + batch)

        #if batch % 100 == 0:
            #loss, current = loss.item(), batch * len(X)


            #pred_df = pd.DataFrame(pred.cpu().detach(), columns=["prediction"])
            #target_df = pd.DataFrame(y.cpu().detach(), columns=["target"])

            #uuid_df = pd.DataFrame(data={"uuid": [uuid.uuid4() for _ in range(len(pred_df.index))]})

            #pred_df = pd.concat([uuid_df, pred_df], axis=1)
            #target_df = pd.concat([uuid_df, target_df], axis=1)

            #amex_metric_value = amex_metric(target_df, pred_df)

            #, amex-score: {amex_metric_value:>7f}

            #print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")     
    return loss.item()

In [ ]:
train_test_ration = 0.9
full_dataset = AmericanExpressProfileTimeSeriesDataset("transformed_dataset.csv", nrows=10000)#, transformation=lambda data: data.T)

train_size = int(train_test_ration * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

#train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
#test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=shuffle)
#validate_dataloader = DataLoader(test_dataset, batch_size=math.floor(test_size/2), shuffle=shuffle)

In [37]:
train_test_ration = 0.7
full_dataset = AmericanExpressPreprocessedProfileTimeSeriesDataset("./amex_preprocessed.npz", nrows=5000)

train_size = int(train_test_ration * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

#train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
#test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=shuffle)
#validate_dataloader = DataLoader(test_dataset, batch_size=math.floor(test_size/2), shuffle=shuffle)

In [38]:
def objective(trial):
    model = ConvNet(trial).to(device)
    loss_fn = nn.BCELoss()
    lr = trial.suggest_float("lr", 1e-7, 0.1, log=True)
    optim_ = trial.suggest_categorical("optimizer", ["AdamW", "Adam", "SGD", "Adagrad", "NAdam"])
    optimizer = getattr(optim, optim_)(model.parameters(), lr=lr)
    lr_lambda = trial.suggest_float("lr_lambda", 0.4, 0.99)
    
    batch_size = trial.suggest_int("batch_size", 16, 128, 16)


    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    scheduler = torch.optim.lr_scheduler.MultiplicativeLR(optimizer, lr_lambda=lambda epoch: lr_lambda)

    epochs = trial.suggest_int("epochs", 1, 3)

    for t in range(epochs):
    
        train_loop(train_dataloader, model, loss_fn, optimizer, t)
        loss = test_loop(test_dataloader, model, loss_fn, t)

        trial.report(loss, t)
        scheduler.step()

    if trial.should_prune():
        raise optuna.exceptions.TrialPruned()

    return loss

In [39]:
study = optuna.create_study(study_name="optuna-study-convolution", direction="minimize", storage='sqlite:///optuna-study.db', load_if_exists=True)

[I 2022-09-17 12:58:04,133] A new study created in RDB with name: optuna-study-convolution


In [41]:
study.optimize(objective, n_trials=1000)

[I 2022-09-17 12:58:22,083] Trial 1 finished with value: 0.702185332775116 and parameters: {'activation_conv': 'LeakyReLU', 'activation_lin': 'LeakyReLU', 'n_layers_blocks': 27, 'conv_size_layer0': 290, 'kernel_size_layer0': 7, 'conv_batchnorm_layer0': True, 'conv_size_layer1': 170, 'kernel_size_layer1': 6, 'conv_batchnorm_layer1': True, 'conv_size_layer2': 170, 'kernel_size_layer2': 11, 'conv_batchnorm_layer2': True, 'conv_size_layer3': 210, 'kernel_size_layer3': 4, 'conv_batchnorm_layer3': False, 'conv_size_layer4': 230, 'kernel_size_layer4': 5, 'conv_batchnorm_layer4': False, 'conv_size_layer5': 200, 'kernel_size_layer5': 2, 'conv_batchnorm_layer5': True, 'conv_size_layer6': 220, 'kernel_size_layer6': 9, 'conv_batchnorm_layer6': False, 'conv_size_layer7': 230, 'kernel_size_layer7': 12, 'conv_batchnorm_layer7': False, 'conv_size_layer8': 130, 'kernel_size_layer8': 11, 'conv_batchnorm_layer8': True, 'conv_size_layer9': 150, 'kernel_size_layer9': 9, 'conv_batchnorm_layer9': True, 'conv

KeyboardInterrupt: 

In [42]:
print("Best trial:")
trial = study.best_trial
print(" Value: ", trial.value)
print(" Params: ")
for key, value in trial.params.items():
    print("     {}: {}".format(key, value))

Best trial:
 Value:  0.2796013961945261
 Params: 
     activation_conv: ReLU
     activation_lin: ReLU
     batch_size: 112
     conv_batchnorm_layer0: False
     conv_batchnorm_layer1: True
     conv_batchnorm_layer2: False
     conv_batchnorm_layer3: True
     conv_size_layer0: 300
     conv_size_layer1: 200
     conv_size_layer2: 180
     conv_size_layer3: 140
     epochs: 2
     kernel_size_layer0: 9
     kernel_size_layer1: 12
     kernel_size_layer2: 5
     kernel_size_layer3: 6
     linear_batchnorm_layer0: False
     linear_batchnorm_layer1: False
     linear_batchnorm_layer2: False
     linear_batchnorm_layer3: False
     linear_batchnorm_layer4: True
     linear_batchnorm_layer5: False
     linear_batchnorm_layer6: False
     linear_size_layer0: 200
     linear_size_layer1: 220
     linear_size_layer2: 260
     linear_size_layer3: 280
     linear_size_layer4: 100
     linear_size_layer5: 130
     linear_size_layer6: 230
     lr: 0.0013045542577553354
     lr_lambda: 0.8976748

In [82]:
class ConvNetOptimized(nn.Module):
    def __init__(self):
        super(ConvNetOptimized, self).__init__()

        self.blocks = nn.Sequential()
        self.blocks.append(nn.Conv1d(178, 300, kernel_size=9, stride=1, padding="same", padding_mode="replicate"))
        self.blocks.append(nn.ReLU())

        self.blocks.append(nn.Conv1d(300, 200, kernel_size=12, stride=1, padding="same", padding_mode="replicate"))
        self.blocks.append(nn.ReLU())
        self.blocks.append(nn.BatchNorm1d(200))

        self.blocks.append(nn.Conv1d(200, 180, kernel_size=5, stride=1, padding="same", padding_mode="replicate"))
        self.blocks.append(nn.ReLU())

        self.blocks.append(nn.Conv1d(180, 140, kernel_size=6, stride=1, padding="same", padding_mode="replicate"))
        self.blocks.append(nn.ReLU())
        self.blocks.append(nn.BatchNorm1d(140))



        self.linear = nn.Sequential()

        self.linear.append(nn.Linear(140, 200))
        self.linear.append(nn.ReLU())

        self.linear.append(nn.Linear(200, 220))
        self.linear.append(nn.ReLU())

        self.linear.append(nn.Linear(220, 260))
        self.linear.append(nn.ReLU())

        self.linear.append(nn.Linear(260, 280))
        self.linear.append(nn.ReLU())

        self.linear.append(nn.Linear(280, 100))
        self.linear.append(nn.ReLU())

        self.linear.append(nn.Linear(100, 130))
        self.linear.append(nn.ReLU())

        self.linear.append(nn.Linear(130, 230))
        self.linear.append(nn.ReLU())

        self.linear.append(nn.Linear(230, 1))
        self.linear.append(nn.Sigmoid())


    def forward(self, x):
        x = self.blocks(x)
        pooling = nn.MaxPool1d(13)
        x = pooling(x)
        x = x.view(x.shape[0], -1)
        x = self.linear(x)
        return x

In [83]:
model = ConvNetOptimized().to(device)

In [84]:
loss_fn = nn.BCELoss()
lr = 0.0013045542577553354
optimizer = optim.Adam(model.parameters(), lr=lr)
lr_lambda = 0.8976748771328298    
batch_size = 112
scheduler = torch.optim.lr_scheduler.MultiplicativeLR(optimizer, lr_lambda=lambda epoch: lr_lambda)
epochs = 15

In [69]:
def get_amex(model, dataset):

    model = model.cpu()

    pred = model(dataset[:][0]).cpu().detach()

    model = model.to(device)

    y = dataset[:][1].cpu().detach()

    pred_df = pd.DataFrame(pred.cpu().detach(), columns=["prediction"])
    target_df = pd.DataFrame(y.cpu().detach(), columns=["target"])
    uuid_df = pd.DataFrame(data={"uuid": [uuid.uuid4() for _ in range(len(pred_df.index))]})

    pred_df = pd.concat([uuid_df, pred_df], axis=1)
    target_df = pd.concat([uuid_df, target_df], axis=1)

    amex_metric_value = amex_metric(target_df, pred_df)
    return amex_metric_value

In [85]:
def test_loop(dataloader, model, loss_fn, dataset):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            pred = torch.round(pred)
            correct += (pred == y).type(torch.float).sum().item()


    test_loss /= num_batches
    correct /= size

    #print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return test_loss

In [86]:
train_loop_num = 0

In [72]:
train_test_ration = 0.7
full_dataset = AmericanExpressPreprocessedProfileTimeSeriesDataset("./amex_preprocessed.npz")

train_size = int(train_test_ration * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [87]:
def train_loop(dataloader, model, loss_fn, optimizer, writer, test_dataset):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        X = X.to(device)
        y = y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        global train_loop_num
        train_loop_num += 1

        writer.add_scalar('BatchLoss/train', loss, train_loop_num)
        if batch % 100 == 0 and batch != 0:
            test_loss = test_loop(test_dataloader, model, loss_fn, test_dataset)
            writer.add_scalar('Loss/test', test_loss, train_loop_num)




        #if batch % 100 == 0:
            #loss, current = loss.item(), batch * len(X)


            #pred_df = pd.DataFrame(pred.cpu().detach(), columns=["prediction"])
            #target_df = pd.DataFrame(y.cpu().detach(), columns=["target"])

            #uuid_df = pd.DataFrame(data={"uuid": [uuid.uuid4() for _ in range(len(pred_df.index))]})

            #pred_df = pd.concat([uuid_df, pred_df], axis=1)
            #target_df = pd.concat([uuid_df, target_df], axis=1)

            #amex_metric_value = amex_metric(target_df, pred_df)

            #, amex-score: {amex_metric_value:>7f}

            #print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")     
    return loss.item()

In [88]:
models = {}

In [89]:
for t in range(epochs):
    writer = SummaryWriter()
    train_loop(train_dataloader, model, loss_fn, optimizer, writer, test_dataset)
    models[t] = model
    scheduler.step()

KeyboardInterrupt: 

In [90]:
model = models[6]

In [91]:
validate_dataset = AmericanExpressPreprocessedProfileTimeSeriesDatasetValidate("./amex_preprocessed.npz")
validate_dataloader = DataLoader(validate_dataset, batch_size=batch_size, shuffle=True)

In [96]:
def validate_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    f1 = torchmetrics.F1Score().to(device)
    precision = torchmetrics.Precision(num_classes=2).to(device)
    recall = torchmetrics.Recall(num_classes=2).to(device)

    all_preds = []
    all_ys = []

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            all_preds.append(pred)
            all_ys.append(y)
            pred = torch.round(pred)
            f1.update(torch.tensor(pred, dtype=torch.int16).view(-1), torch.tensor(y, dtype=torch.int16).view(-1))
            precision.update(torch.tensor(pred, dtype=torch.int16).view(-1), torch.tensor(y, dtype=torch.int16).view(-1))
            recall.update(torch.tensor(pred, dtype=torch.int16).view(-1), torch.tensor(y, dtype=torch.int16).view(-1))

            correct += (pred == y).type(torch.float).sum().item()


    test_loss /= num_batches
    correct /= size

    f1_score = f1.compute()
    precision = precision.compute()
    recall = recall.compute()
    amex = get_amex(torch.Tensor(all_preds), torch.tensor(all_ys))

    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    print("precision", precision)
    print("recall", recall)
    print("f1", f1_score)

In [97]:
validate_loop(validate_dataloader, model, loss_fn)

/tmp/ipykernel_457068/18755555.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  f1.update(torch.tensor(pred, dtype=torch.int16).view(-1), torch.tensor(y, dtype=torch.int16).view(-1))
/tmp/ipykernel_457068/18755555.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  precision.update(torch.tensor(pred, dtype=torch.int16).view(-1), torch.tensor(y, dtype=torch.int16).view(-1))
/tmp/ipykernel_457068/18755555.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  recall.update(torch.tensor(pred, dtype=torch.int16).view(-1), torch.tensor(y, d

AttributeError: 'NoneType' object has no attribute 'append'